<a href="https://colab.research.google.com/github/applejxd/colaboratory/blob/master/LightGbmSample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LightGBM で回帰タスクをするサンプル
- [ボストンの住宅価格に関する回帰タスク](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html)
- [説明変数をDataFrame形式で用意](https://scikit-learn.org/stable/datasets/index.html#boston-dataset)

In [1]:
import pandas as pd
from sklearn.datasets import load_boston

data = load_boston()
train_x: pd.DataFrame = pd.DataFrame(data.data, columns = data.feature_names)

print(len(train_x))
display(train_x.head())

506


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


目的変数も DataFrame 形式で用意

In [2]:
train_y: pd.DataFrame = pd.DataFrame(data.target, columns = ["MEDV"])
display(train_y.head())

,MEDV
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


学習を行ってモデルを戻す関数を定義

In [3]:
# !pip install optuna
# import optuna.integration.lightgbm as lgb
import lightgbm as lgb

def Fit(tr_x: pd.DataFrame, tr_y: pd.Series,
        va_x: pd.DataFrame, va_y: pd.Series) -> lgb.Booster:
    """
    :param tr_x: 学習用説明変数
    :param tr_y: 学習用目的変数
    :param va_x: バリデーション用説明変数
    :param va_y: バリデーション用目的変数
    :return: 学習済みモデル
    """
    # LightGBM が扱う形式に変換
    lgb_train: lgb.Dataset = lgb.Dataset(tr_x, tr_y)
    lgb_eval: lgb.Dataset = lgb.Dataset(va_x, va_y)

    # ハイパーパラメータ
    params = {'objective': 'regression', 'metrics': 'l1', 'seed': 71}

    # 学習の実施
    booster = lgb.train(params, lgb_train, 
                        valid_names=['train', 'valid'],
                        valid_sets=[lgb_train, lgb_eval],
                        verbose_eval=False)
    return booster

学習の実施

In [4]:
# 学習・バリデーションデータの分離
tr_x: pd.DataFrame = train_x.iloc[:380]
va_x: pd.DataFrame = train_x.iloc[381:]
tr_y: pd.DataFrame = train_y.iloc[:380]
va_y: pd.DataFrame = train_y.iloc[381:]

# 学習
model: lgb.Booster = Fit(tr_x, tr_y, va_x, va_y)

# 特徴量評価
importance = pd.DataFrame(model.feature_importance(),
                          index=tr_x.columns.values, 
                          columns=["importance"])
display(importance)

,importance
CRIM,152
ZN,15
INDUS,63
CHAS,2
NOX,115
RM,208
AGE,140
DIS,167
RAD,37
TAX,83


バリデーションの実施

In [5]:
import numpy as np
from sklearn.metrics import mean_absolute_error

va_pred: np.ndarray = model.predict(va_x)
score: float = mean_absolute_error(va_y, va_pred)
print(f'MAE: {score:.4f}')

MAE: 3.9712


クロスバリデーションの準備

In [6]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=4, shuffle=True, random_state=71)

クロスバリデーション実施

In [7]:
scores = []
for tr_idx, va_idx in kf.split(train_x):
    # 学習・バリデーションデータの分離
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # 学習
    model = Fit(tr_x, tr_y, va_x, va_y)

    # バリデーション
    va_pred = model.predict(va_x)
    score = mean_absolute_error(va_y, va_pred)
    scores.append(score)

評価

In [8]:
print(f'MAE (average): {np.mean(scores):.4f}')

MAE (average): 2.2818
